# Prepare (required)

In [ ]:
#@markdown ## Save Mode

saveMode = "googleDrive" #@param ["temporary", "googleDrive"] {"type": "string"}
savePath = ""
if saveMode.lower() == "temporary":
  savePath = "/content/"
else:
  savePath = "/content/drive/MyDrive/"

In [ ]:
#@markdown ## Mount to drive
#@markdown ### If you use googleDrive mode run this cell
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
#@markdown ## Create Folder
#@markdown #### Warning: Don't do this step if you already setup a minecraft server in your drive
import os, os.path
cf = input("Area you sure? [Y/N]: ")
if cf.lower() == "y":
  if os.path.exists(savePath + "minecraftServer"):
    print("Folder exists!")
  else:
    print("Creating Folder...")
    os.system(f"mkdir " + savePath + "minecraftServer")
    print("Done!")

  if os.path.exists(savePath + "minecraftServer/java"):
    print("Folder exists!")
  else:
    print("Creating Folder...")
    os.system("mkdir " + savePath + "minecraftServer/java")
    print("Done!")

  if os.path.exists(savePath + "minecraftServer/bedrock"):
    print("Folder exists!")
  else:
    print("Creating Folder...")
    os.system("mkdir " + savePath + "minecraftServer/bedrock")
    print("Done!")
else:
  print("Ok!")

In [ ]:
#@markdown ## Install requirements
!pip install -U pyngrok
!pip install -U minecraftVersions

In [ ]:
#@markdown ## Ngrok auth token (For tunneling the server) [Required]
import getpass
print("Get auth token from https://dashboard.ngrok.com/get-started/your-authtoken")
ngtoken = getpass.getpass()
with open(savePath+"minecraftServer/ngtoken.txt", "w") as f:
  f.write(ngtoken)

In [ ]:
#@markdown ## Discord Webhook (For announcing server IP) [optional]
import getpass
discordWebhook = getpass.getpass()
with open(savePath+"/minecraftServer/discordWebhook.txt", "w") as f:
  f.write(discordWebhook)

# For Minecraft Java Edition

## 1. Update Java

In [ ]:
#@markdown ## Java 16
%cd /content/
!wget https://files02.tchspt.com/temp/jdk-16.0.2_linux-x64_bin.deb
!sudo apt install /content/jdk-16.0.2_linux-x64_bin.deb
!rm -rf /content/jdk-16.0.2_linux-x64_bin.deb
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/jdk-16.0.2"
!update-alternatives --install /usr/bin/java java /usr/lib/jvm/jdk-16.0.2/bin/java 1919
!java -version

In [ ]:
#@markdown ## Java 17 (Paper 1.18 and above)
%cd /content/
!wget https://download.oracle.com/java/17/archive/jdk-17.0.6_linux-x64_bin.deb
!sudo apt install /content/jdk-17.0.6_linux-x64_bin.deb
!rm -rf /content/jdk-17.0.6_linux-x64_bin.deb
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/jdk-17"
!update-alternatives --install /usr/bin/java java /usr/lib/jvm/jdk-17/bin/java 1919
!java -version

In [ ]:
#@markdown ## Java 19 (Latest version)
%cd /content/
!wget https://download.oracle.com/java/19/latest/jdk-19_linux-x64_bin.deb
!sudo apt install /content/jdk-19_linux-x64_bin.deb
!rm -rf /content/jdk-19_linux-x64_bin.deb
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/jdk-19"
!update-alternatives --install /usr/bin/java java /usr/lib/jvm/jdk-19/bin/java 1919
!java -version

## 3. Download server software (click the arrow to expand)
#### Warning: Don't do this step if you already setup a minecraft server in your drive

In [ ]:
#@markdown ## 2. Set config
#@markdown ### Run the cell to change the config
import ipywidgets as widgets
from IPython.display import display
import minecraftVersions as MCVersion
import requests

serverType = "paper" #@param ["vanilla", "paper"]

mc = MCVersion.MCVersion()

javaServerDownloadUrl = mc.latest().release().server().url
vanillaVersionGui = widgets.Dropdown(
    options=mc.getAllVersions().toList(),
    value=mc.latest().release().version,
    description='MC Version:',
)

paperlist = requests.get("https://papermc.io/api/v2/projects/paper").json()["versions"]
paperVersion = paperlist[-1]
paperVersionGui = widgets.Dropdown(
    options=paperlist,
    value=paperVersion,
    description='PaperMC Version:',
)

saveButton = widgets.Button(description="Save")
def save_config(*arg):
  global vanillaVersionGui
  global javaServerDownloadUrl
  global paperVersion
  javaServerDownloadUrl = mc.getVersion(vanillaVersionGui.value).server().url
  paperVersion = paperVersionGui.value
  print("Saved")

saveButton.on_click(save_config)

if serverType == "vanilla":
  display(vanillaVersionGui)
else:
  display(paperVersionGui)
display(saveButton)


In [ ]:
#@markdown ## Vanilla

import os, os.path
if os.path.exists(savePath + "minecraftServer/java/vanilla"):
  print("Folder exists!")
else:
  print("Creating Folder...")
  os.system("mkdir " + savePath + "minecraftServer/java/vanilla")
  print("Downloading java server...")
  os.system(f"wget {javaServerDownloadUrl} -O {savePath}/minecraftServer/java/vanilla/server.jar")
  print("Done!")

In [ ]:
#@markdown ## PaperMC

import os, os.path
import json
import requests

res = requests.get("https://api.papermc.io/v2/projects/paper/versions/{}/builds/".format(paperVersion))
latestBuild = res.json()["builds"][-1]
paperDownloadUrl = "https://api.papermc.io/v2/projects/paper/versions/{}/builds/{}/downloads/{}".format(paperVersion, latestBuild["build"], latestBuild["downloads"]["application"]["name"])
if os.path.exists(savePath + "minecraftServer/java/paper"):
  print("Folder exists!")
else:
  print("Creating Folder...")
  os.system("mkdir " + savePath + "minecraftServer/java/paper")
  print("Downloading java server...")
  os.system(f"wget {paperDownloadUrl} -O {savePath}minecraftServer/java/paper/server.jar")
  print("Done!")

## 4. Start the server (Java)

In [ ]:
import pyngrok
from pyngrok import ngrok, conf
import getpass
import requests

%cd /content/

def getData(path):
  try:
    return open(path, "r").read().replace("\n", "")
  except:
    return ""

def getServerProp(fp,p):
    with open(fp, "r") as f:
        for i in f.read().splitlines():
            if i[0] != "#":
                k,v = i.split("=")
                if p == k:
                    return v

webhookUrl = getData(savePath + "minecraftServer/discordWebhook.txt")

auth_token = getData(savePath + "minecraftServer/ngtoken.txt")
if auth_token:
  !ngrok authtoken "$auth_token"
else:
  raise ValueError("Failed to get ngrok token!")

serverRegion = "ap" #@param ["us", "eu", "ap", "au", "sa", "jp", "in"] {type: "string"}
conf.get_default().region = serverRegion

software = "paper" #@param ["vanilla", "paper"] {type: "string"}
memory = 10 #@param {type: "slider", min: 1, max: 11, step: 1}

try:
  serverPort = int(getServerProp(f"{savePath}minecraftServer/java/{software}/server.properties", "server-port"))
except:
  serverPort = 25565

client = ngrok.connect(serverPort, 'tcp')
serverip = ((str(client).split('"')[1::2])[0]).replace('tcp://', '')
print("Server IP: " + serverip)

%cd $savePath"/minecraftServer/java/"$software
with open(f"{savePath}minecraftServer/java/{software}/eula.txt", "w") as f:
  f.write("eula=true")
  f.close()

if webhookUrl:
  sendWebhook = requests.post(webhookUrl, json={
    "username": "Server Status",
    "content": "**Server đã mở! Server IP:** `{}`".format(serverip)
  })
  print("Successfully sent the IP to the discord server!" if sendWebhook.status_code == 204 else "Failed to send the IP to the discord server! [{}]".format(sendWebhook.status_code))

import subprocess as sub
import threading

server = sub.Popen(["java", f"-Xms{memory}G", f"-Xmx{memory}G", "-jar", f"{savePath}minecraftServer/java/{software}/server.jar", "nogui"], stdin=sub.PIPE, stdout=sub.PIPE, stderr=sub.PIPE,text=True)
def printLog():
  global server
  while True:
    out = server.stdout.readline()
    if out == '' and server.poll() is not None:
      break
    if out:
      print(out.strip())

threading.Thread(target=printLog).start()
while True:
  try:
    cmd = input()
    if cmd.lower() == "stop":
      server.stdin.write("stop\n")
      server.stdin.flush()
      break
    else:
      server.stdin.write(cmd+"\n")
      server.stdin.flush()
  except KeyboardInterrupt:
    server.stdin.write("stop\n")
    server.stdin.flush()
    break

# For Minecraft Bedrock Edition (Currently ngrok is not support UDP port so this will not work)

In [ ]:
#@markdown ## 1. Download bedrock server
#@markdown ### Current: 1.19.50.02
%cd /content/
bedrockServerDownloadUrl = "https://minecraft.azureedge.net/bin-linux/bedrock-server-1.19.50.02.zip" #@param {type: "string"}
!wget $bedrockServerDownloadUrl -O /content/bedrockServer.zip
!mkdir $savePath"minecraftServer/bedrock/vanilla"
!unzip bedrockServer.zip -d $savePath"minecraftServer/bedrock/vanilla"
!rm -rf /content/bedrockServer.zip



In [ ]:
#@markdown ## 2. Start The Server (Bedrock)
import pyngrok
from pyngrok import ngrok, conf
import getpass

def getData(path):
  try:
    return open(path, "r").read().replace("\n", "")
  except:
    return ""

webhookUrl = getData(savePath + "minecraftServer/discordWebhook.txt")

conf.get_default().region = "ap"
print(ngrok.connect(19132, 'udp'))

%cd "$savePath"minecraftServer/bedrock/vanilla
!LD_LIBRARY_PATH=. ./bedrock_server

# Tools

In [ ]:
#@markdown ## Quick add plugin (Paper)

import os, os.path

pluginUrl = "https://mediafilez.forgecdn.net/files/3348/374/OpenInv.jar" #@param {type: "string"}

def downloadPlugin():
  global pluginUrl
  if pluginUrl:
    print("Fetching data from {}".format(pluginUrl))
    os.system(f"wget {pluginUrl} -P {savePath}minecraftServer/java/paper/plugins/")
    print(f"Done! Saved to {savePath}minecraftServer/java/paper/plugins")
  else:
    print("Invaild URL!")

if os.path.exists(savePath + "minecraftServer/java/paper/plugins"):
  downloadPlugin()
else:
  os.system("mkdir " + savePath + "minecraftServer/java/paper/plugins")
  downloadPlugin()


Fetching data from https://mediafilez.forgecdn.net/files/3348/374/OpenInv.jar
Done! Saved to /content/drive/MyDrive/minecraftServer/java/paper/plugins
